In [ ]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/')
sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl')
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.loaders.mr_us_dataset as mr_us_dataset
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed


In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis'

In [ ]:
# model_fn = os.path.join(mount_point, 'train_output/simu_reconstruction/USPCReconstruction/v0.2/epoch=7-val_loss=0.01.ckpt')
model_fn = os.path.join(mount_point, 'train_output/simu_reconstruction/USPCReconstruction/v0.3/epoch=8-val_loss=0.01.ckpt')
pc_r = us_simu.USPCReconstruction.load_from_checkpoint(model_fn)
pc_r.eval()
pc_r.cuda()

In [ ]:
diffusor_np, diffusor_head = nrrd.read('/mnt/raid//C1_ML_Analysis/simulated_data_export/placenta/FAM-025-0664-4_label11_resampled.nrrd')
diffusor_t = torch.tensor(diffusor_np.astype(int)).permute(2, 1, 0)
print(diffusor_head)
diffusor_size = torch.tensor(diffusor_head['sizes'])
diffusor_spacing = torch.tensor(np.diag(diffusor_head['space directions']))

diffusor_origin = torch.tensor(diffusor_head['space origin']).flip(dims=[0])
diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size
print(diffusor_spacing)
print(diffusor_t.shape)
print(diffusor_origin)
print(diffusor_end)

In [ ]:
batch_size = 1
diffusor_batch_t = diffusor_t.permute([2, 1, 0]).cuda().float().unsqueeze(0).unsqueeze(0).repeat(batch_size, 1, 1, 1, 1)
diffusor_origin_batch = diffusor_origin[None, :].repeat(batch_size, 1)
diffusor_end_batch = diffusor_end[None, :].repeat(batch_size, 1)

print(diffusor_batch_t.shape, diffusor_origin_batch.shape, diffusor_end_batch.shape)

In [ ]:

X_V = []
with torch.no_grad():
    for t_idx, tag in enumerate(pc_r.vs.tags):
        V, F = pc_r.get_sweeps(diffusor_batch_t.cuda(), diffusor_origin_batch.cuda(), diffusor_end_batch.cuda(), train=False, tag_idx=t_idx)

        x_s, _ = pc_r(F, V)

        threshold = np.percentile(x_s.detach().cpu().numpy(), 95)
        x_s = x_s.reshape(-1)
        V = V.reshape(-1, 3)
        x_v = V[x_s >= threshold]
        X_V.append(x_v)
X_V = torch.cat(X_V, dim=0)
X_V = torch.unique(X_V, dim=0)


In [ ]:
x_v = X_V.cpu().numpy()
fig = go.Figure(data=[go.Scatter3d(x=x_v[:,0], y=x_v[:,1], z=x_v[:,2], mode='markers', marker=dict(
        size=2,
        color=x_v[:,2],                # set color to an array/list of desired values
        colorscale='jet',   # choose a colorscale
        opacity=1.0
    ))])
fig.show()